# YOLO Tennis Ball Tracking Inference

This notebook demonstrates the inference process using a trained YOLOv8 model for tennis ball detection and tracking. The notebook includes two main approaches:

1. **Simple Tracking**: Basic YOLO tracking with built-in visualization
2. **Custom Trajectory Visualization**: Advanced tracking with custom trajectory drawing and confidence filtering

## Overview
- **Model**: YOLOv8m trained on tennis ball dataset for 150 epochs
- **Input**: Video frames from test clips
- **Output**: Annotated videos with tennis ball detection and trajectory visualization
- **Features**: Confidence thresholding, trajectory trails, and persistent tracking

## Import Required Libraries

Import essential libraries for YOLO model inference and video processing:
- `tqdm`: Progress bar for visual feedback during processing
- `ultralytics.YOLO`: YOLOv8 model class for object detection and tracking
- `cv2`: OpenCV for computer vision operations and video handling

In [ ]:
from tqdm import tqdm
from ultralytics import YOLO
import cv2

## Method 1: Basic YOLO Tracking

This section demonstrates the simplest approach to tennis ball tracking using YOLO's built-in tracking capabilities.

**Features:**
- Loads the trained YOLOv8m model (150 epochs)
- Performs object tracking with persistence across frames
- Displays real-time tracking results
- Automatically saves annotated output video

**Parameters:**
- `persist=True`: Maintains object IDs across frames
- `show=True`: Displays tracking in real-time window
- `save=True`: Saves annotated video output

In [ ]:
# Load Model and Test Clip paths
model = YOLO('runs/detect/yolov8m_tennis_ballv2_150epochs/weights/best.pt')
video_path = 'videos/Clip1.mp4'

# Run tracking 
# 'persist=True' tells the tracker to remember objects between frames
# 'save=True' will save the output video with tracking annotations
results = model.track(
    source=video_path,
    show=True, 
    save=True,
    persist=True
)


WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/207) c:\Users\jake\dev\IP_A2\TracknetImplementation\Tennis-Ball-Tracking\src\trackers\yolo\videos\Clip1_temp.mp4: 384x640 (no detections), 52.5ms
video 1/1 (frame 2/207) c:\Users\jake\dev\IP_A2\TracknetImplementation\Tennis-Ball-Tracking\src\trackers\yolo\videos\Clip1_temp.mp4: 384x640 1 tennis_ball, 7.5ms
video 1/1 (frame 3/207) c:\Users\jake\dev\IP_A2\TracknetImplementation\Tennis-Ball-Tracking\src\trackers\yolo\videos\Clip1_temp.mp4:

## Method 2: Advanced Trajectory Visualization

This section implements a more sophisticated tracking approach with custom trajectory visualization and confidence filtering.

**Key Features:**
- **Frame-to-Video Conversion**: Creates temporary video from individual frame images
- **Confidence Thresholding**: Only tracks detections above 30% confidence
- **Trajectory Trails**: Draws continuous path of tennis ball movement
- **Best Detection Selection**: Chooses highest confidence detection per frame
- **Custom Visualization**: Overlays trajectory trail on YOLO annotations

**Workflow:**
1. Convert frame images to temporary video
2. Run YOLO tracking with streaming results
3. Filter detections by confidence threshold
4. Calculate ball center points for trajectory
5. Draw cumulative trajectory trail
6. Save final annotated video with trajectory

In [ ]:
from pathlib import Path

def create_video_from_frames(frames_dir, output_path, fps):
    """
    Convert a directory of frame images into a single video file.
    
    Args:
        frames_dir (str): Path to directory containing .jpg frame files
        output_path (str): Output path for the compiled video file
        fps (int): Frames per second for the output video
        
    Returns:
        tuple: (success_flag, video_properties)
            - success_flag (bool): True if video creation succeeded
            - video_properties (tuple or None): (width, height, frame_count) if successful, None if failed
    """
    # Get sorted list of all .jpg frame files
    frame_files = sorted(Path(frames_dir).glob('*.jpg'))
    if not frame_files:
        print(f"Error: No .jpg frames found in {frames_dir}")
        return False, None
    
    # Read first frame to determine video dimensions
    first_frame = cv2.imread(str(frame_files[0]))
    height, width, _ = first_frame.shape
    
    # Initialize video writer with MP4 codec
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    # Compile all frames into video
    print(f"Compiling {len(frame_files)} frames into temporary video...")
    for frame_file in tqdm(frame_files, desc="Compiling Video"):
        frame = cv2.imread(str(frame_file))
        video_writer.write(frame)
    
    # Release video writer and return success status
    video_writer.release()
    print("Temporary video compilation complete.")
    return True, (width, height, len(frame_files))

# ============================================================================
# CONFIGURATION PARAMETERS
# ============================================================================

# Model configuration
MODEL_PATH = 'runs/detect/yolov8m_tennis_ballv2_150epochs/weights/best.pt'
model = YOLO(MODEL_PATH)

# Input/Output paths
INPUT_FRAMES_DIR = 'videos/Clip1'
TEMP_VIDEO_PATH = 'videos/Clip1.mp4'
OUTPUT_VIDEO_PATH = 'videos/Clip1_yolo_trajectory.mp4'
OUTPUT_FPS = 30

# Detection parameters
CONF_THRESHOLD = 0.3  # Minimum confidence score for valid detections

# Trajectory visualization settings
TRAIL_COLOR = (255, 0, 0)  # BGR format: Red trajectory line
TRAIL_THICKNESS = 3

# ============================================================================
# VIDEO PROCESSING PIPELINE
# ============================================================================

# Step 1: Convert frame images to temporary video file
success, video_props = create_video_from_frames(INPUT_FRAMES_DIR, TEMP_VIDEO_PATH, OUTPUT_FPS)
if not success:
    exit()

# Step 2: Initialize video writer for output with trajectory overlay
original_width, original_height, total_frames = video_props
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, OUTPUT_FPS, (original_width, original_height))

# Initialize trajectory storage for ball position history
trajectory_points = []

# Step 3: Run YOLO tracking with streaming results for memory efficiency
results = model.track(source=TEMP_VIDEO_PATH, stream=True, persist=True)

print(f"Processing video and drawing trail with confidence threshold of {CONF_THRESHOLD}...")

# Process each frame for detection and trajectory drawing
for r in tqdm(results, total=total_frames):
    # Get YOLO annotated frame (with bounding boxes)
    annotated_frame = r.plot()

    # Process detections if any boxes found in current frame
    if len(r.boxes) > 0:
        # Extract detection data from YOLO results
        boxes = r.boxes.xyxy.cpu().numpy().astype(int)  # Bounding box coordinates
        confidences = r.boxes.conf.cpu().numpy()        # Confidence scores
        
        # Select highest confidence detection in current frame
        best_box_idx = confidences.argmax()
        
        # Filter detection by confidence threshold
        if confidences[best_box_idx] >= CONF_THRESHOLD:
            # Calculate center coordinates of best detection
            best_box = boxes[best_box_idx]
            center_x = (best_box[0] + best_box[2]) // 2  # (x1 + x2) / 2
            center_y = (best_box[1] + best_box[3]) // 2  # (y1 + y2) / 2
            
            # Add validated center point to trajectory history
            trajectory_points.append((center_x, center_y))

    # Draw trajectory trail connecting all historical points
    if len(trajectory_points) > 1:
        for i in range(1, len(trajectory_points)):
            cv2.line(annotated_frame, 
                    trajectory_points[i - 1], 
                    trajectory_points[i], 
                    TRAIL_COLOR, 
                    TRAIL_THICKNESS)
    
    # Write frame with overlaid trajectory to output video
    video_writer.write(annotated_frame)

# ============================================================================
# CLEANUP AND COMPLETION
# ============================================================================

# Release video writer and close OpenCV windows
video_writer.release()
cv2.destroyAllWindows()

# Display completion status
print(f"\n--- Processing Complete ---")
print(f"Final trajectory video saved to: {OUTPUT_VIDEO_PATH}")
print(f"Total trajectory points tracked: {len(trajectory_points)}")

Compiling 207 frames into temporary video...


Compiling Video: 100%|██████████| 207/207 [00:00<00:00, 245.46it/s]


Temporary video compilation complete.
Processing video and drawing trail with confidence threshold of 0.3...


  0%|          | 0/207 [00:00<?, ?it/s]


video 1/1 (frame 1/207) c:\Users\jake\dev\IP_A2\TracknetImplementation\Tennis-Ball-Tracking\src\trackers\yolo\videos\Clip1.mp4: 384x640 (no detections), 48.0ms


...

video 1/1 (frame 202/207) c:\Users\jake\dev\IP_A2\TracknetImplementation\Tennis-Ball-Tracking\src\trackers\yolo\videos\Clip1.mp4: 384x640 1 tennis_ball, 8.0ms
video 1/1 (frame 203/207) c:\Users\jake\dev\IP_A2\TracknetImplementation\Tennis-Ball-Tracking\src\trackers\yolo\videos\Clip1.mp4: 384x640 1 tennis_ball, 8.2ms
video 1/1 (frame 204/207) c:\Users\jake\dev\IP_A2\TracknetImplementation\Tennis-Ball-Tracking\src\trackers\yolo\videos\Clip1.mp4: 384x640 1 tennis_ball, 8.5ms
video 1/1 (frame 205/207) c:\Users\jake\dev\IP_A2\TracknetImplementation\Tennis-Ball-Tracking\src\trackers\yolo\videos\Clip1.mp4: 384x640 1 tennis_ball, 9.4ms


 99%|█████████▉| 205/207 [00:05<00:00, 39.14it/s]

video 1/1 (frame 206/207) c:\Users\jake\dev\IP_A2\TracknetImplementation\Tennis-Ball-Tracking\src\trackers\yolo\videos\Clip1.mp4: 384x640 2 tennis_balls, 9.0ms
video 1/1 (frame 207/207) c:\Users\jake\dev\IP_A2\TracknetImplementation\Tennis-Ball-Tracking\src\trackers\yolo\videos\Clip1.mp4: 384x640 (no detections), 9.0ms
Speed: 0.8ms preprocess, 7.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)


100%|██████████| 207/207 [00:05<00:00, 39.36it/s]


--- Processing Complete ---
Final trajectory video saved to: videos/Clip1_yolo_trajectory.mp4


### Configuration Parameters

**Model Settings:**
- `MODEL_PATH`: Path to the trained YOLOv8m model weights
- `CONF_THRESHOLD`: Minimum confidence score (0.3) for valid detections

**Input/Output Paths:**
- `INPUT_FRAMES_DIR`: Directory containing individual frame images
- `TEMP_VIDEO_PATH`: Temporary video file created from frames
- `OUTPUT_VIDEO_PATH`: Final output video with trajectory overlay
- `OUTPUT_FPS`: Frame rate for output video (30 FPS)

**Visualization Settings:**
- `TRAIL_COLOR`: RGB color for trajectory line (red: 255,0,0)
- `TRAIL_THICKNESS`: Line thickness for trajectory visualization

### Processing Pipeline

**Step 1: Video Preparation**
- Compiles individual frame images into a temporary video file
- Validates frame availability and determines video dimensions

**Step 2: Detection and Tracking**
- Processes video frame-by-frame using YOLO tracking
- Filters detections based on confidence threshold
- Selects best detection per frame (highest confidence)
- Calculates center coordinates for trajectory points

**Step 3: Trajectory Visualization**
- Draws cumulative trajectory trail connecting all valid points
- Overlays trail on YOLO's annotated frames
- Saves final video with both detection boxes and trajectory path